## Andrew Byrnes: Fetch Rewards Coding Exercise - Data Analyst
### 2.1-EDA_first_pass.ipynb

This notebook represents a first draft / draft as I explore the exercise questions. It does not neccessiarly contain my final answers, but is for the most part reflective of how I would start to approach these problems. My final presentation of answers can be found in 2.2-FINAL-ANSWERS.ipynb.

### Data Sources
- fetch.db: SQLite file from notebook 1-Data_Prep
- fetch_db_erd.jpg: ERD image, documenting fetch.db schema designed and created in notebook 1-Data_Prep

### Changes
- 09-21-2022 : Set up notebook
- 09-22-2022 : 

In [1]:
import pandas as pd
from pathlib import Path
import os
from datetime import datetime
import sqlite3

### File Locations

In [5]:
today = datetime.today()
print(today)
db_path = Path.cwd() / "data" / "processed" / "fetch.db"
erd_fetch_db = Path.cwd() / "data" / "interim" / "fetch_db_erd.jpg" 

2022-09-22 11:25:26.195513


### Formatting and options

In [23]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows')
pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_columns')

## First: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

1-Data_Prep.ipynb capatures my process from start to finsih of reviewing the data, performing some transfomrations, and loading 4 tables of data into a SQLite database. 

I've structured the data as presented in this ERD:
  
*Note: brands.barcode <-> receipt_items is not implemented as a 0 or 1 to zero or many relationship in fetch.db as depicted above. There are duplicate values present in brands.barcode, those values are flagged in brands.dupe_barcode for easy filtering. Without filtering this join may cause unintended duplicates and in a production environment would need to be addressed.*

## Second: Write a query that directly answers a predetermined question from a business stakeholder

In [ ]:
query = f"""
        select 
            
        from
            
        where
            
    """
#execute the above query and save results to dataframe 
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()
df

###  What are the top 5 brands by receipts scanned for most recent month?
I don't want to over count receipts for a brand if multiple different items of the same brand were purchased on the same receipt. Is that a possiblity with this data and something I need to account for?

Question: are there duplicate brand names in the brands table? / is barcode 1to1 with product? / can a brand have multiple barcodes(products) on the same receipt? if so, does that receipt count once or twice?
- there are 10 brands with 2 unique barcodes each. often across different categories, but not always

Assumptions: 
- by receipts scanned: Only count a receipt once, even if it contains purchases from two distinct barcodes assoicated with the same band or two of the same barcode purchsed multiple times on the same receipt. We'll want to count unqiue receipt ids as 'scanned receipts'

- Recent Month: It looks like the receipts data was capatured on 3/1/21 around 11pm. I'm going to assume 'most recent month' refers to the most recent complete month. I'm assuming a complete month can be indicated by the preseence of values in the following month. In this case there is some data from March, so I'll assume February is the most recent, complete, month. With 444 records in February and 637 in January it seems safe to assume those are complete sets, realtive to the other three months present. 

In [77]:
# Q: Does a brands.name appear multiple times in the brands table? (Exclude known dupe barcodes)
# A: Yes, for ten brands. They each have 2 distinct barcodes

query = f"""
        select 
            b.name,
            count(b.barcode) as barcodes
        from
            brands b
        where
            b.dupe_barcode is not True
        group by
            b.name
        having
            -- only return results where the count of barcode is greater than 1
            barcodes > 1
        order by
            name
            
    """
#execute the above query and save results to dataframe 
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()
df

name  barcodes
0                         Baken-Ets         2
1                      Caleb's Kola         2
2          Diabetic Living Magazine         2
3               Dippin Dots® Cereal         2
4                           Huggies         2
5  I CAN'T BELIEVE IT'S NOT BUTTER!         2
6                 ONE A DAY® WOMENS         2
7                          Pull-Ups         2
8                       Sierra Mist         2
9                        V8 Hydrate         2

In [78]:
"""Q: Beyond Name are there other duplicate fields for these records?
Look at the results of the brands table for all brads.name that appear mutiple times
A: It doesn't appear to be a consistent pattern - topBrand seens inconsistent, i would expect both or none
of a brand with multiple records to have topBrand values to be True or False, often there is both a True and False entry
"""

query = f"""
        with brands_w_multiple_barcodes as(
            select 
                b.name,
                count(b.barcode) as barcodes
            from
                brands b
            where
                b.dupe_barcode is not True
            group by
                b.name
            having
                -- only return results where the count of barcode is greater than 1
                barcodes > 1
            order by
                barcodes desc
        )
        
        select 
            *
        from
            brands b
        where
            b.name in  (select name from brands_w_multiple_barcodes)
        order by
            name
            
    """
#execute the above query and save results to dataframe 
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()
df

id       barcode dupe_barcode              category  \
0   5d9d08d1a60b87376833e348  511111605546         None                Snacks   
1   585a961fe4b03e62d1ce0e76  511111701781         None                Snacks   
2   5a4d23dae4b0bcb2c74ea77e  511111000518         None             Beverages   
3   5d601d74a3a018514994f422  511111004912         None                Snacks   
4   5d66d597a3a018093ab34726  511111805298         None             Magazines   
5   5d658ffa6d5f3b23d1bc7914  511111205227         None                  None   
6   5dc2d9d4a60b873d6b0666d2  511111206330         None    Breakfast & Cereal   
7   5dc1fca91dda2c0ad7da64ae  511111706328         None    Breakfast & Cereal   
8   5bd2011f90fa074576779a17  511111704652         None                  Baby   
9   5c7d9cb395144c337a3cbfbb  511111707202         None                  Baby   
10  592486bee410d61fcea3d12d  511111700814         None                 Dairy   
11  5e710dd5ee7f2d0b35b2a193  511111314097         None  Dairy & Refrigerated   
12  5da609991dda2c3e1416ae90  511111805854         None     Health & Wellness   
13  5e5ff265ee7f2d0b35b2a18f  511111914051         None     Health & Wellness   
14  5bd201a990fa074576779a19  511111104698         None                  Baby   
15  5db3288aee7f2d6de4248977  511111312949         None                  Baby   
16  5332f608e4b03c9a25efd0c1  511111903901         None                  None   
17  5d66e07da3a018093ab3472d  511111205500         None             Beverages   
18  5bcdfc5a965c7d66d92731e9  511111304616         None             Beverages   
19  5bcdfc5990fa074576779a15  511111804604         None             Beverages   

              categoryCode                       cpg  \
0                     None  5332f5fbe4b03c9a25efd0ba   
1                     None  5332f5fbe4b03c9a25efd0ba   
2                     None  5332f5fbe4b03c9a25efd0ba   
3                     None  53e10d6368abd3c7065097cc   
4                     None  5d5d4fd16d5f3b23d1bc7905   
5                     None  53e10d6368abd3c7065097cc   
6                     None  5332f5f3e4b03c9a25efd0ae   
7                     None  53e10d6368abd3c7065097cc   
8                     None  550b2565e4b001d5e9e4146f   
9                     BABY  5459429be4b0bfcb1e864082   
10                    None  53e10d6368abd3c7065097cc   
11  DAIRY_AND_REFRIGERATED  5e5ea9cc9c50da1291a6d074   
12                    None  53e10d6368abd3c7065097cc   
13                    None  53e10d6368abd3c7065097cc   
14                    None  550b2565e4b001d5e9e4146f   
15                    None  550b2565e4b001d5e9e4146f   
16                    None  53e10d6368abd3c7065097cc   
17                    None  5332f5fbe4b03c9a25efd0ba   
18                    None  53e10d6368abd3c7065097cc   
19                    None  5a734034e4b0d58f376be874   

                                name  topBrand  \
0                          Baken-Ets       NaN   
1                          Baken-Ets       1.0   
2                       Caleb's Kola       0.0   
3                       Caleb's Kola       NaN   
4           Diabetic Living Magazine       NaN   
5           Diabetic Living Magazine       NaN   
6                Dippin Dots® Cereal       NaN   
7                Dippin Dots® Cereal       NaN   
8                            Huggies       0.0   
9                            Huggies       1.0   
10  I CAN'T BELIEVE IT'S NOT BUTTER!       NaN   
11  I CAN'T BELIEVE IT'S NOT BUTTER!       NaN   
12                 ONE A DAY® WOMENS       0.0   
13                 ONE A DAY® WOMENS       NaN   
14                          Pull-Ups       0.0   
15                          Pull-Ups       1.0   
16                       Sierra Mist       NaN   
17                       Sierra Mist       NaN   
18                        V8 Hydrate       NaN   
19                        V8 Hydrate       0.0   

                           brandCode  
0                          BAKEN ETS  
1                 

In [32]:
# getting a sense of receipt scanned volume across different time frames - figure out how to define 'most recent month'
# generate some examples of SQL date formatting 
# ref: SQLite Date and Time Functions: https://www.sqlite.org/lang_datefunc.html
query = f"""
        select 
            -- dateScanned,
            -- date(dateScanned),
            date(dateScanned, 'start of month'),
            -- date(dateScanned, 'start of month', '-1 month'),
            count(*)
        from
            receipts
        group by
            1 
        order by 
            1 desc
        """
#execute the above query and save results to dataframe 
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()
df

date(dateScanned, 'start of month')  count(*)
0                          2021-03-01        30
1                          2021-02-01       444
2                          2021-01-01       637
3                          2020-11-01         6
4                          2020-10-01         2

In [34]:
# subquery to determine 'most recent month'
query = f"""
        select
            max(dateScanned),
            date(max(dateScanned), 'start of month')
        from
            receipts
        
            
        """
#execute the above query and save results to dataframe 
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()
df

max(dateScanned) date(max(dateScanned), 'start of month')
0  2021-03-01 23:17:34.772000                               2021-03-01

In [100]:
# Answer
query = f"""
        select 
            b.name,
            count(ri.receipt_id) as receipts_scanned
        from
            brands b
            join receipt_items ri on coalesce(ri.barcode, ri.userFlaggedBarcode) = b.barcode
            join receipts r on r.id = ri.receipt_id
        where
            -- b.dupe_barcode is not True
            -- and 
            date(r.dateScanned) >= '2021-02-01'
                                       /*
                                       (
                                        select
                                            -- the start of the most recent month of data
                                            date(max(dateScanned), 'start of month', '-1 month')
                                        from
                                            receipts
                                        )
                                        */
                                        
        group by
            1
        order by
            2 desc
        limit 5            
        """
#execute the above query and save results to dataframe 
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()
df

Empty DataFrame
Columns: [name, receipts_scanned]
Index: []

In [93]:
# troubleshooting why i'm not getting results
query = f"""
        select 
            dateScanned
            , date(dateScanned)
        from
            receipts
        where
            date(dateScanned) >=  (
                                        select
                                            -- the start of the most recent month of data
                                            date(max(dateScanned), 'start of month', '-1 month')
                                        from
                                            receipts
                                        )
        order by
            1 desc
            
    """
#execute the above query and save results to dataframe 
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()
df

dateScanned date(dateScanned)
0    2021-03-01 23:17:34.772000        2021-03-01
1           2021-03-01 21:32:28        2021-03-01
2           2021-03-01 21:17:27        2021-03-01
3           2021-03-01 19:27:33        2021-03-01
4           2021-03-01 18:22:30        2021-03-01
5    2021-03-01 17:47:34.867000        2021-03-01
6    2021-03-01 15:42:41.873000        2021-03-01
7    2021-03-01 14:07:37.664000        2021-03-01
8    2021-03-01 13:57:34.396000        2021-03-01
9           2021-03-01 13:07:28        2021-03-01
10          2021-03-01 11:52:29        2021-03-01
11   2021-03-01 11:47:35.734000        2021-03-01
12          2021-03-01 10:32:28        2021-03-01
13          2021-03-01 10:22:27        2021-03-01
14          2021-03-01 10:02:29        2021-03-01
15          2021-03-01 09:57:28        2021-03-01
16   2021-03-01 09:02:35.175000        2021-03-01
17   2021-03-01 08:57:33.696000        2021-03-01
18   2021-03-01 08:22:34.244000        2021-03-01
19   2021-03-01 07:57:34.307000        2021-03-01
20   2021-03-01 06:42:39.190000        2021-03-01
21          2021-03-01 06:22:28        2021-03-01
22          2021-03-01 05:32:28        2021-03-01
23   2021-03-01 04:17:35.736000        2021-03-01
24   2021-03-01 03:52:38.407000        2021-03-01
25   2021-03-01 03:12:35.733000        2021-03-01
26   2021-03-01 02:27:34.329000        2021-03-01
27   2021-03-01 02:22:34.962000        2021-03-01
28   2021-03-01 01:57:34.494000        2021-03-01
29          2021-03-01 01:02:28        2021-03-01
30          2021-02-28 23:22:27        2021-02-28
31          2021-02-28 21:27:30        2021-02-28
32   2021-02-28 20:07:35.488000        2021-02-28
33   2021-02-28 19:52:35.205000        2021-02-28
34   2021-02-28 19:37:34.389000        2021-02-28
35          2021-02-28 18:07:29        2021-02-28
36          2021-02-28 17:22:31        2021-02-28
37          2021-02-28 17:07:28        2021-02-28
38   2021-02-28 14:42:35.401000        2021-02-28
39          2021-02-28 13:27:27        2021-02-28
40   2021-02-28 13:22:35.358000        2021-02-28
41   2021-02-28 11:27:34.288000        2021-02-28
42   2021-02-28 11:17:35.369000        2021-02-28
43          2021-02-28 10:57:27        2021-02-28
44          2021-02-28 08:07:27        2021-02-28
45   2021-02-28 07:27:37.125000        2021-02-28
46   2021-02-28 06:27:36.453000        2021-02-28
47          2021-02-28 05:37:30        2021-02-28
48   2021-02-28 05:12:35.950000        2021-02-28
49   2021-02-28 04:22:36.166000        2021-02-28
50   2021-02-28 04:12:37.474000        2021-02-28
51          2021-02-28 03:17:30        2021-02-28
52          2021-02-28 02:12:30        2021-02-28
53   2021-02-28 01:52:38.608000        2021-02-28
54   2021-02-28 01:27:35.090000        2021-02-28
55          2021-02-28 01:27:28        2021-02-28
56   2021-02-28 01:22:36.150000        2021-02-28
57          2021-02-28 01:02:29        2021-02-28
58          2021-02-28 00:12:27        2021-02-28
59   2021-02-28 00:02:34.325000        2021-02-28
60   2021-02-27 22:07:34.763000        2021-02-27
61   2021-02-27 21:47:34.253000        2021-02-27
62          2021-02-27 20:27:27        2021-02-27
63   2021-02-27 20:02:34.525000        2021-02-27
64   2021-02-27 18:32:35.382000        2021-02-27
65          2021-02-27 17:52:31        2021-02-27
66   2021-02-27 17:47:35.915000        2021-02-27
67   2021-02-27 17:27:37.226000        2021-02-27
68   2021-02-27 16:27:34.465000        2021-02-27
69   2021-02-27 16:17:37.078000        2021-02-27
70          2021-02-27 15:47:27        2021-02-27
71          2021-02-27 14:17:27        2021-02-27
72   2021-02-27 13:22:37.305000        2021-02-27
73   2021-02-27 13:17:35.009000        2021-02-27
74   2021-02-27 12:22:34.453000        2021-02-27
75          2021-02-27 11:52:27        2021-02-27
76          2021-02-27 11:17:27        2021-02-27
77   2021-02-27 10:42:35.264000        2021-02-27
78   2021-02-27 09:52:34.507000        2021-02-27
79   2021-02-27 08:5

In [104]:
# joining to brands breaks things, I wonder if there is something wrong with barcodes after 2021-01-25
query = f"""
        select 
            r.id
            , ri.receipt_id
            , dateScanned
            , date(dateScanned)
            , ri.barcode
            , ri.userFlaggedBarcode
            , b.name
        from
            receipts r
            join receipt_items ri on ri.receipt_id = r.id
            join brands b on b.barcode = coalesce(ri.barcode, ri.userFlaggedBarcode)
        order by
            3 desc
            
    """
#execute the above query and save results to dataframe 
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()
df

id                receipt_id          dateScanned  \
0   600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
1   600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
2   600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
3   600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
4   600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
5   600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
6   600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
7   600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
8   600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
9   600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
10  600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
11  600f39c30a7214ada2000030  600f39c30a7214ada2000030  2021-01-25 21:36:03   
12  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
13  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
14  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
15  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
16  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
17  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
18  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
19  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
20  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
21  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
22  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
23  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
24  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
25  600f2fc80a720f0535000030  600f2fc80a720f0535000030  2021-01-25 20:53:28   
26  600edb570a720f053500001d  600edb570a720f053500001d  2021-01-25 14:53:11   
27  600eda0c0a7214ada200001e  600eda0c0a7214ada200001e  2021-01-25 14:47:40   
28  600ba6ae0a7214ada2000010  600ba6ae0a7214ada2000010  2021-01-23 04:31:42   
29  600b420b0a7214ada200000d  600b420b0a7214ada200000d  2021-01-22 21:22:19   
30  600b420b0a7214ada200000d  600b420b0a7214ada200000d  2021-01-22 21:22:19   
31  600b420b0a7214ada200000d  600b420b0a7214ada200000d  2021-01-22 21:22:19   
32  600b420b0a7214ada200000d  600b420b0a7214ada200000d  2021-01-22 21:22:19   
33  600b420b0a7214ada200000d  600b420b0a7214ada200000d  2021-01-22 21:22:19   
34  600b420b0a7214ada200000d  600b420b0a7214ada200000d  2021-01-22 21:22:19   
35  600b420b0a7214ada200000d  600b420b0a7214ada200000d  2021-01-22 21:22:19   
36  600b39f60a720f053500000e  600b39f60a720f053500000e  2021-01-22 20:47:50   
37  600b139e0a7214ada200000a  600b139e0a7214ada200000a  2021-01-22 18:04:13   
38  600afbd90a720f053500000b  600afbd90a720f053500000b  2021-01-22 16:22:49   
39  600af82f0a720f053500000a  600af82f0a720f053500000a  2021-01-22 16:07:11   
40  600af82f0a720f053500000a  600af82f0a720f053500000a  2021-01-22 16:07:11   
41  600a1e270a720f0535000009  600a1e270a720f0535000009  2021-01-22 00:36:55   
42  600a1e270a720f0535000009  600a1e270a720f0535000009  2021-01-22 00:36:55   
43  600a1e270a720f0535000009  600a1e270a720f0535000009  2021-01-22 00:36:55   
44  600a1e270a720f0535000009  600a1e270a720f0535000009  2021-01-22 00:36:55   
45  600a1a8d0a7214ada2000008  600a1a8d0a7214ada2000008  2021-01-22 00:21:32   
46  600a1a8d0a7214ada2000008  600a1a8d0a7214ada2000008  2021-01-22 00:21:32   
47  600a1a8d0a7214ada2000008  600a1a8d0a7214ada2000008  2021-01-22 00:21:32   
48  6009f5ea0a7214ada2000004  6009f5ea0a7214ada2000004  2021-01-21 21:45:14   
49  6008ee0e0a7214ad89000125  6008ee0e0a7214ad89000125  2021-01-21 02:59:26 

In [105]:
#check out what barcodes look like on receipt_items for receipts scanned after 2021-01-25
# barcodes are in a different format after around 2/26/21, then not joining to brands

query = f"""
        select 
            r.id
            , date(r.dateScanned)
            , ri.barcode
            , ri.userFlaggedBarcode
        from
            receipts r
            join receipt_items ri on ri.receipt_id = r.id
        order by 
            2 desc
            
    """
#execute the above query and save results to dataframe 
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()
df

id date(r.dateScanned)        barcode  \
0     603cbbb50a720fde100003e3          2021-03-01     B076FJ92M4   
1     603cbbb50a720fde100003e3          2021-03-01     B07BRRLSVC   
2     603d30e60a7217c72c00043f          2021-03-01     B076FJ92M4   
3     603d30e60a7217c72c00043f          2021-03-01     B07BRRLSVC   
4     603d5d6c0a7217c72c000463          2021-03-01     B076FJ92M4   
5     603d5d6c0a7217c72c000463          2021-03-01     B07BRRLSVC   
6     603d59e70a7217c72c00045f          2021-03-01     B076FJ92M4   
7     603d59e70a7217c72c00045f          2021-03-01     B07BRRLSVC   
8     603d40250a720fde10000459          2021-03-01     B076FJ92M4   
9     603d40250a720fde10000459          2021-03-01     B07BRRLSVC   
10    603c88240a7217c72c0003b8          2021-03-01     B076FJ92M4   
11    603c88240a7217c72c0003b8          2021-03-01     B07BRRLSVC   
12    603cba880a7217c72c0003e0          2021-03-01     B076FJ92M4   
13    603cba880a7217c72c0003e0          2021-03-01     B07BRRLSVC   
14    603cd57d0a7217c72c0003f6          2021-03-01     B076FJ92M4   
15    603cd57d0a7217c72c0003f6          2021-03-01     B07BRRLSVC   
16    603c7c6c0a7217c72c0003b3          2021-03-01     B076FJ92M4   
17    603c7c6c0a7217c72c0003b3          2021-03-01     B07BRRLSVC   
18    603c3d240a720fde10000373          2021-03-01     B076FJ92M4   
19    603c3d240a720fde10000373          2021-03-01     B07BRRLSVC   
20    603cc2bc0a720fde100003e9          2021-03-01     B076FJ92M4   
21    603cc2bc0a720fde100003e9          2021-03-01     B07BRRLSVC   
22    603cc0630a720fde100003e6          2021-03-01     B076FJ92M4   
23    603cc0630a720fde100003e6          2021-03-01     B07BRRLSVC   
24    603ce7100a7217c72c000405          2021-03-01     B076FJ92M4   
25    603ce7100a7217c72c000405          2021-03-01     B07BRRLSVC   
26    603b9a3f0a720fde100002d3          2021-02-28     B076FJ92M4   
27    603b9a3f0a720fde100002d3          2021-02-28     B07BRRLSVC   
28    603af1800a7217c72c00027b          2021-02-28     B076FJ92M4   
29    603af1800a7217c72c00027b          2021-02-28     B07BRRLSVC   
30    603c25b30a720fde1000035b          2021-02-28     B076FJ92M4   
31    603c25b30a720fde1000035b          2021-02-28     B07BRRLSVC   
32    603b2c1a0a7217c72c0002a9          2021-02-28     B076FJ92M4   
33    603b2c1a0a7217c72c0002a9          2021-02-28     B07BRRLSVC   
34    603bd1570a7217c72c000337          2021-02-28     B076FJ92M4   
35    603bd1570a7217c72c000337          2021-02-28     B07BRRLSVC   
36    603bcdd00a7217c72c000335          2021-02-28     B076FJ92M4   
37    603bcdd00a7217c72c000335          2021-02-28     B07BRRLSVC   
38    603b0b4a0a720fde1000024e          2021-02-28     B076FJ92M4   
39    603b0b4a0a720fde1000024e          2021-02-28     B07BRRLSVC   
40    603bdbe10a7217c72c00033e          2021-02-28     B076FJ92M4   
41    603bdbe10a7217c72c00033e          2021-02-28     B07BRRLSVC   
42    603b77170a7217c72c0002e6          2021-02-28     B076FJ92M4   
43    603b77170a7217c72c0002e6          2021-02-28     B07BRRLSVC   
44    603b4f3f0a7217c72c0002c9          2021-02-28     B076FJ92M4   
45    603b4f3f0a7217c72c0002c9          2021-02-28     B07BRRLSVC   
46    603c0ac20a7217c72c00035c          2021-02-28     B076FJ92M4   
47    603c0ac20a7217c72c00035c          2021-02-28     B07BRRLSVC   
48    603afc0e0a720fde10000243          2021-02-28     B076FJ92M4   
49    603afc0e0a720fde10000243          2021-02-28     B07BRRLSVC   
50    603adfeb0a7217c72c000265          2021-02-28     B076FJ92M4   
51    603adfeb0a7217c72c000265          2021-02-28     B07BRRLSVC   
52    603aeba50a7217c72c000272          2021-02-28     B076FJ92M4   
53    603aeba50a7217c72c000272          2021-02-28     B07BRRLSVC   
54    603a2a470a720fde10000197          2021-02-27     B076FJ92M4   
55    603a2a470a720fde10000197          2021-02-27     B07BRRLSVC   
56    603a327b0a7217c72c0001d1          2021-02-27     B076FJ92M4   
57    603a327b0a7217c72c00

In [109]:
#examine receipts.rewardsReceiptItemList for some of the sample receipts with non consistent barcode fomrats
query = f"""
        select 
            id,
            rewardsReceiptItemList
        from
            receipts
        where
            id in ('603cbbb50a720fde100003e3', '603d30e60a7217c72c00043f', '603d5d6c0a7217c72c000463', '603ce7100a7217c72c000405')
    """
#execute the above query and save results to dataframe 
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()
df

id  \
0  603cbbb50a720fde100003e3   
1  603ce7100a7217c72c000405   
2  603d30e60a7217c72c00043f   
3  603d5d6c0a7217c72c000463   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     rewardsReceiptItemList  
0  [{'barcode': 'B076FJ92M4', 'description': 'mueller austria hypergrind precision electric spice/coffee grinder millwith large grinding capacity and hd motor also for spices, herbs, nuts,grains, white', 'discountedItemPrice': '22.97', 'finalPrice': '22.97', 'itemPrice': '22.97', 'originalReceiptItemText': 'mueller austria hypergrind precision electric spice/coffee grinder millwith large grinding capacity and hd motor also for spices, herbs, nuts,grains, white', 'partnerItemId': '0', 'priceAfterCoupon': '22.97', 'quantityPurchased': 1}, {'barcode': 'B07BRRLSVC', 'description': 'thindust summer face mask - sun protection neck gaiter for outdooractivities', 'discountedItemPrice': '11.99', 'finalPrice': '11.99', 'itemPrice': '11.99', 'originalReceiptItemText': 'thindust summer face mask - sun protection neck gaiter for outdooractivities', 'partnerItemId': '1', 'priceAfterCoupon': '11.99', 'quantityPurchased': 1}]  
1  [{'barcode': 'B076FJ92M4', 'description': 'mueller austria hypergrind precision electric spice/coffee grinder millwith large grinding capacity and hd motor also for spices, herbs, nuts,grains, white', 'discountedItemPrice': '22.97', 'finalPrice': '22.97', 'itemPrice': '22.97', 'originalReceiptItemText': 'mueller austria hypergrind precision electric spice/coffee grinder millwith large grinding capacity and hd motor also for spices, herbs, nuts,grains, white', 'partnerItemId': '0', 'priceAfterCoupon': '22.97', 'quantityPurchased': 1}, {'barcode': 'B07BRRLSVC', 'description': 'thindust summer face mask - sun protection neck gaiter for outdooractivities', 'discountedItemPrice': '11.99', 'finalPrice': '11.99', 'itemPrice': '11.99', 'originalReceiptItemText': 'thindust summer face mask - sun protection neck gaiter for outdooractivities', 'partnerItemId': '1', 'priceAfterCoupon': '11.99', 'quantityPurchased': 1}]  
2  [{'barcode': 'B076FJ92M4', 'description': 'mueller austria hypergrind precision electric spice/coffee grinder millwith large grinding capacity and hd motor also for spices, herbs, nuts,grains, white', 'discountedItemPrice': '22.97', 'finalPrice': '22.97', 'itemPrice': '22.97', 'originalReceiptItemText': 'mueller austria hypergrind precision electric spice/coffee grinder millwith large grinding capacity and hd motor also for spices, herbs, nuts,grains, white', 'partnerItemId': '0', 'priceAfterCoupon': '22.97', 'quantityPurchased': 1}, {'barcode': 'B07BRRLSVC', 'description': 'thindust summer face mask - sun protection neck gaiter for outdooractivities', 'discountedItemPrice': '11.99', 'finalPrice': '11.99', 'itemPrice': '11.99', 'originalReceiptItemText': 'thindust summer face mask - sun protection neck gaiter for outdooractivities', 'partnerItemId': '1', 'priceAfterCoupon': '11.99', 'quantityPurchased': 1}]  
3  [{'barcode': 'B076FJ92M4', 'description': 'mueller austria hypergrind precision electric spice/coffee grinder millwith large grinding capacity and hd motor also f

In [67]:
# looking into brands.barcode
query = f"""
        select 
            dupe_barcode
        from
            brands
        where 
            
        
            
    """
#execute the above query and save results to dataframe 
conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)
conn.close()
df

dupe_barcode
0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
5              NaN
6              NaN
7              NaN
8              NaN
9              1.0
10             NaN
11             NaN
12             NaN
13             NaN
14             NaN
15             NaN
16             NaN
17             NaN
18             NaN
19             NaN
20             1.0
21             NaN
22             NaN
23             NaN
24             NaN
25             NaN
26             NaN
27             NaN
28             NaN
29             NaN
30             NaN
31             NaN
32             NaN
33             NaN
34             NaN
35             NaN
36             NaN
37             NaN
38             NaN
39             NaN
40             NaN
41             NaN
42             NaN
43             NaN
44             NaN
45             NaN
46             NaN
47             NaN
48             NaN
49             NaN
50             NaN
51             NaN
52             NaN
53             NaN
54             NaN
55             NaN
56             NaN
57             NaN
58             NaN
59             NaN
60             NaN
61             NaN
62             NaN
63             NaN
64             NaN
65             NaN
66             NaN
67             NaN
68             NaN
69             NaN
70             NaN
71             NaN
72             NaN
73             NaN
74             NaN
75             NaN
76             NaN
77             NaN
78             NaN
79             NaN
80             NaN
81             NaN
82             NaN
83             NaN
84             NaN
85             NaN
86             NaN
87             NaN
88             NaN
89             NaN
90             NaN
91             NaN
92             NaN
93             NaN
94             NaN
95             NaN
96             NaN
97             NaN
98             NaN
99             NaN
100            NaN
101            NaN
102            NaN
103            NaN
104            NaN
105            NaN
106            NaN
107            NaN
108            NaN
109            NaN
110            NaN
111            NaN
112            NaN
113            NaN
114            NaN
115            NaN
116            NaN
117            NaN
118            NaN
119            NaN
120            NaN
121            NaN
122            NaN
123            NaN
124            NaN
125            NaN
126            NaN
127            NaN
128            NaN
129            1.0
130            NaN
131            NaN
132            NaN
133            NaN
134            NaN
135            NaN
136            NaN
137            NaN
138            NaN
139            NaN
140            NaN
141            NaN
142            NaN
143            NaN
144            NaN
145            NaN
146            NaN
147            NaN
148            NaN
149            NaN
150            NaN
151            NaN
152            1.0
153            NaN
154            NaN
155            NaN
156            NaN
157            NaN
158            NaN
159            NaN
160            NaN
161            NaN
162            NaN
163            NaN
164            NaN
165            NaN
166            NaN
167            NaN
168            NaN
169            NaN
170            NaN
171            NaN
172            NaN
173            NaN
174            NaN
175            NaN
176            NaN
177            NaN
178            NaN
179            NaN
180            NaN
181            NaN
182            NaN
183            NaN
184            NaN
185            NaN
186            NaN
187            NaN
188            NaN
189            NaN
190            NaN
191            NaN
192            NaN
193            NaN
194            1.0
195            NaN
196            NaN
197            NaN
198            NaN
199            NaN
200            NaN
201            NaN
202            NaN
203            NaN
204            NaN
205            NaN
206            NaN
207            NaN
208            NaN
209            N

### How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

### Which brand has the most spend among users who were created within the past 6 months?

### Which brand has the most transactions among users who were created within the past 6 months?

## Third: Evaluate Data Quality Issues in the Data Provided

Using the programming language of your choice (SQL, Python, R, Bash, etc...) identify at least one data quality issue. We are not expecting a full blown review of all the data provided, but instead want to know how you explore and evaluate data of questionable provenance.

## Fourth: Communicate with Stakeholders

Construct an email or slack message that is understandable to a product or business leader who isn’t familiar with your day to day work. This part of the exercise should show off how you communicate and reason about data with others. Commit your answers to the git repository along with the rest of your exercise.

- What questions do you have about the data?
- How did you discover the data quality issues?
- What do you need to know to resolve the data quality issues?
- What other information would you need to help you optimize the data assets you're trying to create?
- What performance and scaling concerns do you anticipate in production and how do you plan to address them?